In [1]:
from IPython.display import Markdown as md

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
import sys

# setting path
sys.path.append('./CUB_Code/')
import findSequenceById as FSB
import pandas as pd
import numpy as np

def createBinaryList(lst):
    binaryList=[]
    med=np.median(lst)
    for x in lst:
        if x>=med:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList


def convertListToLabels(lst,n_label=3,distri=[0.2,0.6,0.2],labels=[0,1,2]):
    sortedLst=lst.sorted()
    




geneDict=FSB.findSequenceByID("./Data/s288c.fasta")
df1=pd.DataFrame(geneDict.items(),columns=["GeneName","Sequence"])
df2=pd.read_csv("./S288C_Expression/EmpiricalExp/exp.csv")
df=pd.merge(df1,df2,on="GeneName")
binList=createBinaryList(df["EXP"])
df["Label"]=binList
df.to_csv("Full_S288C.datatable.csv")


#Create a samll table where middle exp tiers are removed and provide strong exp signals (higher confidence on labels)
topSelectionRatio=1
botSelectionRatio=1
topSize=int(topSelectionRatio*len(df))
botSize=int(botSelectionRatio*len(df))

lowIndex=botSize
highIndex=len(df)-topSize

#sort the table by exp, and remove the "middle" hunk
df=df.sort_values("EXP")
df=df.drop(df.index[lowIndex:highIndex])



Selected id Type: locus_tag
There are 0 entries NOT found out of 5990
5990 distinct record in 5990 entries


After loading the data, the next step is to convert the raw coding sequences to text sentences compsed of codon usage ranks.

In [4]:
#Start

In [5]:
from Bio import SeqIO
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './CUB_Code')
import CodonLibraries as CL

seqList=df["Sequence"]
sentenceList=[]
for seq in seqList:
    codonList = CL.loadSequence(seq)
    sentence=""
    for codon in codonList:
        sentence+=codon+" "
    sentenceList.append(sentence[:-1])#this removes the extra redundant space in the end

Now we have converted the sequence to codonLists, then concated them into string sentences 

In [6]:
print(sentenceList[:1])

['AAC TAT TTG GAA ACA CAG TTA AAT AAA AAG CAA AAA CAG ATA CAG GAA TAC GAA AGT ATG AAT GGC AAC CTG ATA AAG ATG TTT GAG CAA TTG TCT AAA GAA AAG AAA AAT GAT GAG ACA CCA AAA AAA ATT TCC TCG ACG TAC ATT AAA GAG TTA AAG GAG TAC AAC GAA TTG AGA GAT GCC GGT TTA AGG TTG GCC CAA ATA ATT GCT GAT GAA AAG CAA TGC AAA ATT AAG GAT GTT TTT GAA GAG ATC GGT TAT TCA ATG AAG GAC']


The next step is to get the genome wide codon usage and generate codon ranks, which will used to replace the codon strings

In [7]:
import findSequenceById as FSBID
from CAI import RSCU
import scipy.stats as ss




def process(seqList,genomeFile,tag=""):
    geneDict=FSBID.findSequenceByID(genomeFile)
    keyList=[]
    rscu=RSCU(seqList)
    rscu_rank=convertRSCUtoRanks(rscu)
    sentenceList = []
    for seq in seqList:
        codonList=CL.loadSequence(seq)
        #remove the first and last five codons:
        codonList = codonList[5:]
        codonList = codonList[:-5]
        try:
            codonRankList=[rscu_rank[codon] for codon in codonList]
            sentence=""
            for rank in codonRankList:
                sentence+=str(rank)+" "
            sentenceList.append(sentence[:-1])
        except :
            print("one error on ",seq)
    
    return sentenceList




def convertRSCUtoRanks(rscu):
    synonymousCodons = {
        'CYS': ['TGT', 'TGC'],
        'ASP': ['GAT', 'GAC'],
        'SER': ['TCT', 'TCG', 'TCA', 'TCC', 'AGC', 'AGT'],
        'GLN': ['CAA', 'CAG'],
        'MET': ['ATG'],
        'ASN': ['AAC', 'AAT'],
        'PRO': ['CCT', 'CCG', 'CCA', 'CCC'],
        'LYS': ['AAG', 'AAA'],
        'THR': ['ACC', 'ACA', 'ACG', 'ACT'],
        'PHE': ['TTT', 'TTC'],
        'ALA': ['GCA', 'GCC', 'GCG', 'GCT'],
        'GLY': ['GGT', 'GGG', 'GGA', 'GGC'],
        'ILE': ['ATC', 'ATA', 'ATT'],
        'LEU': ['TTA', 'TTG', 'CTC', 'CTT', 'CTG', 'CTA'],
        'HIS': ['CAT', 'CAC'],
        'ARG': ['CGA', 'CGC', 'CGG', 'CGT', 'AGG', 'AGA'],
        'TRP': ['TGG'],
        'VAL': ['GTA', 'GTC', 'GTG', 'GTT'],
        'GLU': ['GAG', 'GAA'],
        'TYR': ['TAT', 'TAC']}#'CYS': ['TGT', 'TGC']
    rscu_rank=dict()
    for aa in synonymousCodons:
        codonList=synonymousCodons[aa]
        rscuList=[rscu[codon] for codon in codonList]
        rankList=ss.rankdata([-1*x for x in rscuList])
        for codon,rank in zip(codonList,rankList):
            rscu_rank[codon]=int(rank)
    return rscu_rank



In [8]:
genomeFile="./Data/s288c.fasta"
print(len(seqList))
print(seqList[:10])


4568
1941    ATGAACTATTTGGAAACACAGTTAAATAAAAAGCAAAAACAGATAC...
4329    ATGCACAATCAGGAAGAGTGGCTAGACAAGGACAAAACTTTGGTGA...
1496    ATGGGAGCTGGCACTCTTTTAAATGGATTAGAAAAGGAAAACTTTC...
2025    ATGAATCAAGGTTACACACAGCTTTCCGCACCGGAACTGAAGGAGA...
4202    ATGCTTTTCGCTAGATTAGTGCTGCTGTTGGTGTATTTGGCACCAG...
3056    ATGGATTGCCCCTCAAACGTTGTGTTATTGTTGCTGCAATTAGTTT...
2788    ATGAACATTAAGACTTTGTGTCATCCAGAATATAAAAGAATCTCCG...
1425    ATGAGTTTCCTAAGCATTTTTACTTTTTTCAGCGTCCTTATTTCTG...
3323    ATGTACGAGTACTGCTCAGTTGTAATAAAGAAATACTCCAAGTATA...
1492    ATGGTGACTGGTGAAGAAAATGTGTATCTAAAGTCAAGCTTATCCA...
Name: Sequence, dtype: object


In [9]:
sentenceList=process(list(seqList),genomeFile)

Selected id Type: locus_tag
There are 0 entries NOT found out of 5990
5990 distinct record in 5990 entries


In [10]:
print(len(sentenceList))
print(sentenceList[:1])

4568
['2 2 1 1 2 1 1 2 2 2 1 2 1 3 1 1 3 2 5 2 2 1 1 2 1 1 1 1 1 2 1 1 1 2 2 1 1 1 1 4 6 4 2 1 1 2 2 2 2 2 2 1 1 1 1 3 1 2 2 1 3 1 2 1 1 1 1 2 1 2 1 1 2 1 1 1 1 2 3 1']


We now have the intermediate sentence structure composed of characters including all codon rnaks, this can also be used for training the model and tokenizer, but we will apply one more operation on this file, composing single codon ranks into words of size k

In [11]:
def convertLineToWords(line,n):
    line= line.replace(' ','')
    wordList=[]
    for i in range(0,len(line)-n,n):
        wordList.append(line[i:i+n])
    sentence=" ".join(wordList)
    return sentence



In [12]:
k=1 #k specifies the wordsize, aka number of codon ranks in a word, or understood as size of codon windows
wordSentenceList=[]
for sentence in sentenceList:
    wordSentence=convertLineToWords(sentence,k)
    wordSentenceList.append(wordSentence)

In [13]:
labelList=df["Label"]



In [14]:
df["WordSentence"]=wordSentenceList

In [15]:
df["SentenceLength"]=[len(x) for x in df["WordSentence"]]
df=df[df['SentenceLength'] >= 2]
print(len(df))

4567


In [16]:
df_sub=df[["WordSentence","Label"]].copy()
df_sub=df_sub.rename(columns={"WordSentence": "text", "Label": "label"})
OutputName=""
OutputName+="./Data/expressionPrediction_S288C"
OutputName+="_top_"+str(topSelectionRatio)
OutputName+="_bot_"+str(botSelectionRatio)
OutputName+="_k_"+str(k)
OutputName+=".csv"
df_sub.to_csv(OutputName)

In [17]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_sub)
df.to_csv("FullTable_k1.csv")

#### 